In [1]:
import pandas as pd
import numpy as np

In [2]:
df_nonsplit = pd.read_csv("alignments.tsv", sep="\t")
df_nonsplit = df_nonsplit[(df_nonsplit["split"] == False) & df_nonsplit["query_cigar"].str.contains('S')]
df_nonsplit.to_csv("special_cases.csv")
pd.set_option('display.max_colwidth', None)
df_nonsplit = df_nonsplit[['split', 'AA_homology_seq', 'break_chrom1', 'break_pos1', 'break_chrom2', 'break_pos2', 'query_name', 'query_chrom', 'query_pos', 'query_end', 'query_orientation', 'query_cigar', 'query_aln_sub', 'query_aln_full']]
df_nonsplit.query_name = df_nonsplit.query_name.apply(lambda x: x.split(':')[-1])
df_nonsplit
nonsplit_matches = df_nonsplit[
    (df_nonsplit["break_pos1"] == df_nonsplit["query_pos"]) | 
    (df_nonsplit["break_pos1"] == df_nonsplit["query_end"]) | 
    (df_nonsplit["break_pos2"] == df_nonsplit["query_pos"]) | 
    (df_nonsplit["break_pos2"] == df_nonsplit["query_end"])
]

In [3]:
# df_nonsplit['break_chrom1'] = df_nonsplit['break_chrom1'].astype(str)

In [4]:
# bs_sv = set(df_nonsplit[df_nonsplit["AA_homology_seq"].isna()]['break_pos1']).union(set(df_nonsplit[df_nonsplit["AA_homology_seq"].isna()]['break_pos2'])).remove(94023408)
# bs_sv
# df2 = pd.read_csv("alignments.tsv", sep="\t")
# thems_the_breaks = set([item for tup in set(list(zip(zip(df2.break_chrom1, df2.break_pos1), zip(df2.break_chrom2, df2.break_pos2)))) for item in tup if item[1] not in bs_sv])
# thems_the_breaks
# def check_break(row):
#     for i in thems_the_breaks:
#         if row['query_chrom'] == i[0] and row['query_pos'] <= i[1] and row['query_end'] >= i[1]:
#             return i
#     return np.nan
# df2['breakpoint'] = df2.apply(check_break, axis=1)
# df2.to_csv('with_breaks.csv')

In [5]:
df_split = pd.read_csv("alignments.tsv", sep="\t")
df_split = df_split[df_split["split"] == True]
pd.set_option('display.max_colwidth', None)
df_split = df_split[['split', 'AA_homology_seq', 'break_chrom1', 'break_pos1', 'break_chrom2', 'break_pos2', 'query_name', 'query_chrom', 'query_pos', 'query_end', 'query_orientation', 'query_cigar', 'query_aln_sub', 'query_aln_full']]
df_split.query_name = df_split.query_name.apply(lambda x: x.split(':')[-1])
df_split
half_match = df_split[
    (df_split["break_pos1"] == df_split["query_pos"]) | 
    (df_split["break_pos1"] == df_split["query_end"]) | 
    (df_split["break_pos2"] == df_split["query_pos"]) | 
    (df_split["break_pos2"] == df_split["query_end"])
]
full_match = half_match[half_match.groupby("query_name")["query_name"].transform("size") > 1]
full_match

,split,AA_homology_seq,break_chrom1,break_pos1,break_chrom2,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full
0,True,GAGTG,chr9,133607147,chr22,23632742,92037,chr9,133607147,133607212,+,34S66M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA
1,True,GAGTG,chr9,133607147,chr22,23632742,92037,chr22,23632704,23632742,+,39M61H,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG
3,True,GAGTG,chr9,133607147,chr22,23632742,31831,chr22,23632682,23632742,-,61M39S,GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA
4,True,GAGTG,chr9,133607147,chr22,23632742,31831,chr9,133607147,133607190,-,56H44M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA
7,True,GAGTG,chr9,133607147,chr22,23632742,88045,chr9,133607147,133607177,-,69H31M,GAGTGGGTTTTATCAGCTTCCATACCCAAAC,GAGTGGGTTTTATCAGCTTCCATACCCAAAC
8,True,GAGTG,chr9,133607147,chr22,23632742,88045,chr22,23632669,23632742,-,74M26S,GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAAC
9,True,GAGTG,chr9,133607147,chr22,23632742,93025,chr9,133607147,133607217,-,29S71M,GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA
10,True,GAGTG,chr9,133607147,chr22,23632742,93025,chr22,23632709,23632742,-,34M66H,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG,ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG
95,True,AT,chr9,134155524,chr13,108661411,29921,chr13,108661342,108661411,+,70M30S,GAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,GAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCTTCTCCTGC
96,True,AT,chr9,134155524,chr13,108661411,29921,chr9,134155493,134155524,-,32M68H,GCAGGAGAAGGGACCAGGAAGCATCAGTCCAT,GCAGGAGAAGGGACCAGGAAGCATCAGTCCAT


In [6]:
new_awesomesauce_df =pd.concat([nonsplit_matches, full_match], axis=0).sort_values(["AA_homology_seq", "split"])
new_awesomesauce_df["break_start"] = (new_awesomesauce_df["break_pos1"] == new_awesomesauce_df["query_pos"]) | (new_awesomesauce_df["break_pos2"] == new_awesomesauce_df["query_pos"])
new_awesomesauce_df

,split,AA_homology_seq,break_chrom1,break_pos1,break_chrom2,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full,break_start
377,True,AA,chr13,92474011,chr13,92475758,57596,chr13,92473942,92474011,-,70M30S,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTT,False
378,True,AA,chr13,92474011,chr13,92475758,57596,chr13,92475727,92475758,+,32M68H,AAATTAATGCTCCCACTTCAACTTAAAGAATT,AAATTAATGCTCCCACTTCAACTTAAAGAATT,False
381,True,AA,chr13,92474011,chr13,92475758,83594,chr13,92475722,92475758,-,37M63H,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,False
382,True,AA,chr13,92474011,chr13,92475758,83594,chr13,92473947,92474011,+,65M35S,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTTACTTT,False
186,True,AAC,chr13,81088033,chr13,81469935,78137,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True
187,True,AAC,chr13,81088033,chr13,81469935,78137,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True
190,True,AAC,chr13,81088033,chr13,81469935,60717,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True
191,True,AAC,chr13,81088033,chr13,81469935,60717,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True
128,False,AT,chr9,134155524,chr13,108661411,77319,chr13,108661338,108661411,+,74M26S,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCTTCTC,False
137,False,AT,chr9,134155524,chr13,108661411,60793,chr13,108661334,108661411,+,78M22S,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCT,False


In [7]:
cooked_df = pd.read_csv("alignments.tsv", sep="\t")
cooked_df = cooked_df[
    (
        (cooked_df['query_chrom'] != cooked_df['break_chrom1']) &
        (cooked_df['query_chrom'] != cooked_df['break_chrom2'])
    ) | 
    ((
        (cooked_df['break_pos1'] > cooked_df['query_end']) |
        (cooked_df['break_pos1'] < cooked_df['query_pos'])
    ) &
    (
        (cooked_df['break_pos2'] > cooked_df['query_end']) |
        (cooked_df['break_pos2'] < cooked_df['query_pos'])
    ))
]
cooked_df = cooked_df.loc[cooked_df['query_cigar'].str.contains('S') | cooked_df['query_cigar'].str.contains('H')].drop(["break_sv_type", "break_read_support", "break_features", "query_aln_full"], axis=1)
cooked_df.to_csv("minor_errors.tsv", sep="\t")

In [8]:
def process_row(row):
    if (row["break_pos1"] == row["query_pos"] or row["break_pos1"] == row["query_end"]):
        if not row["break_pos1"] == row["query_pos"]:
            complement = {'C': 'G', 'G': 'C', 'A': 'T', 'T': 'A'}
            return (''.join([complement[bp] for bp in row["query_aln_sub"]]))[::-1]
        else:
            return row["query_aln_sub"]
    else:
        if not row["break_pos2"] == row["query_end"]:
            complement = {'C': 'G', 'G': 'C', 'A': 'T', 'T': 'A'}
            return (''.join([complement[bp] for bp in row["query_aln_sub"]]))[::-1]
        else:
            return row["query_aln_sub"]

new_awesomesauce_df["new_awesomesauce_query"] = new_awesomesauce_df.apply(process_row, axis=1)
new_awesomesauce_df

,split,AA_homology_seq,break_chrom1,break_pos1,break_chrom2,break_pos2,query_name,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full,break_start,new_awesomesauce_query
377,True,AA,chr13,92474011,chr13,92475758,57596,chr13,92473942,92474011,-,70M30S,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTT,False,TTTGATGATTAGTTAGCATCCATTAATATAATAATCATATGAATAGATTAGAAGAAATCACATTACGTCT
378,True,AA,chr13,92474011,chr13,92475758,57596,chr13,92475727,92475758,+,32M68H,AAATTAATGCTCCCACTTCAACTTAAAGAATT,AAATTAATGCTCCCACTTCAACTTAAAGAATT,False,AAATTAATGCTCCCACTTCAACTTAAAGAATT
381,True,AA,chr13,92474011,chr13,92475758,83594,chr13,92475722,92475758,-,37M63H,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,False,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT
382,True,AA,chr13,92474011,chr13,92475758,83594,chr13,92473947,92474011,+,65M35S,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTTACTTT,False,TTTGATGATTAGTTAGCATCCATTAATATAATAATCATATGAATAGATTAGAAGAAATCACATTA
186,True,AAC,chr13,81088033,chr13,81469935,78137,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG
187,True,AAC,chr13,81088033,chr13,81469935,78137,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAAC
190,True,AAC,chr13,81088033,chr13,81469935,60717,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,True,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAAC
191,True,AAC,chr13,81088033,chr13,81469935,60717,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,True,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG
128,False,AT,chr9,134155524,chr13,108661411,77319,chr13,108661338,108661411,+,74M26S,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCTTCTC,False,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT
137,False,AT,chr9,134155524,chr13,108661411,60793,chr13,108661334,108661411,+,78M22S,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCT,False,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT


In [9]:
first = [
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGG",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATA",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGGTTTCTTTGATCCTAG",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAAC",
    "GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA",
]
last = [
    'CTCCACCCAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
           'CAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
            'AGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                                                             'TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                                       'GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                          'GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
                                                                  'ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG',
]

def mini_homology_inator(str_list, dir):
    str_list = [i[::dir] for i in str_list]
    slice = 0
    for i in range(len(max(str_list, key=len))):
        temp_list = [i[:slice] for i in str_list]
        if not len(set(temp_list)) <= 1:
            break
        slice += 1
    return str_list[0][:slice - 1][::dir]

def homology_inator(first, last):
    first_str = mini_homology_inator(first, 1)
    last_str = mini_homology_inator(last, -1)
    longest_str = ''
    for i in range(min(len(first_str), len(last_str))):
        if first_str[:i] == last_str[-i:]:
            longest_str = first_str[:i]
    print(longest_str)
    return longest_str

homology_inator(first, last)

GAGTG


'GAGTG'

In [10]:
new_awesomesauce_df["first"] = new_awesomesauce_df.apply(lambda row: row["break_pos1"] == row["query_pos"] or row["break_pos1"] == row["query_end"], axis=1)
grouped_queries = new_awesomesauce_df.groupby(["break_pos1", "first"])["new_awesomesauce_query"].agg(list)
grouped_queries = grouped_queries.reset_index().pivot(index="break_pos1", columns="first", values="new_awesomesauce_query")
grouped_queries.columns = ["last", "first"]
grouped_queries['homology'] = grouped_queries.apply(lambda row: homology_inator(row["first"], row["last"]), axis=1)
grouped_queries


AAC
CCA
TTTTAT
TT

GAGTG
AT


,last,first,homology
break_pos1,,,
18953123,[TTAACTAACTCAGCACAAACTCTGAGACCCAA],[TCCATCTGGGAAACGGAACAAACCTGCATGCCGCATGGATCATGGTGGCCCAGAGTGGGGTGCCCAGG],
81088033,"[GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAAC, GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAAC]","[AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG, AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG]",AAC
81109414,"[AAATAGAATTATAATCAAGATGGACCAAAAATAAAGTTAAAAAACTGAGCTTAAGATTTTGCCCA, CAAAAATAAAGTTAAAAAACTGAGCTTAAGATTTTGCCCA]","[CCAGACATATGAGGTAAAATTAATTCAGTCAGTGACAGCTATTATGTAACCAATACATTGCACTGTGGAGATAAACACATAGGCAC, CCAGACATATGAGGTAAAATTAATTCAGTCAGAGACAGCTATTATGTAACCAATACATTGCACTGTGGAGATAAACACA, CCAGACATATGAGGTAAAATTAATTCAGTCAGTGACAGCTATTATGTAACCAATACATTGCACTGTGCAGATAAAC, CCAGACATATGAGGTAAAATTAATTCAGTCAGTGACAGCTATTATGTAACCAATACATTGCACTGTGGAGATAA, CCAGACATATGAGGTAAAATTAATTCAGTCAGTGACAGCTATTATGTAACCAATACATTGCACTGTGGAGATAAACACATA, CCAGACATATGAGGTAAAATTAATTCAGTCAGTGACAG, CCAGACATATGAGGTAAAATTAATTCAGTCAGTGACAGCTATTATGTAACCAATACATTGCAC]",CCA
81470036,"[ATGCCTGAATAATATTGATATTAGTTATTTGTAACTTGTATCTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT, CTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT]",[TTTTATTTCAGACTCATATGGAACATCCGTGTGTATGTGTATAAATAGTTTAT],TTTTAT
92474011,"[AAATTAATGCTCCCACTTCAACTTAAAGAATT, AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT]","[TTTGATGATTAGTTAGCATCCATTAATATAATAATCATATGAATAGATTAGAAGAAATCACATTACGTCT, TTTGATGATTAGTTAGCATCCATTAATATAATAATCATATGAATAGATTAGAAGAAATCACATTA]",TT
94023408,[GGAGAGACACAGAGGATGCTTAGGGAATTTAGGATAGAT],"[AAAATGCATATGTCAAGAACTATGTTACATTTCAACCTAGTTTTCAAAATCAGAATACAACATCTGTCACAACCATCCACATAGCTGTGGACTATGTT, AAAATGCATATGTCAAGAACTATGTTACATTTCAACCTAGTTTTCAAAATCAGAATACAACATCTGTCACAACCATCCACATAGCTG, AAAATGCATATGTCAAGAACTATGTTACATTTCAACCTAGTTTTCAAAATCAGAATACAAC]",
133607147,"[CTCCACCCAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, AGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, CAGGAAGGACTAATCGGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, TGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, GGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, GGGCAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG, ACGGGACACCTTTGACCCTGGCCGCTGTGGAGTG]","[GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGG, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATA, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAATCTATAGGTTTCTTTGATCCTAG, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCA, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTA, GAGTGGGTTTTATCAGCTTCCATACCCAAAC, GAGTGGGTTTTATCAGCTTCCATACCCAAACAGAAATACCCTTAAGGATTTTCTTCTCTGATTGCACTAAA]",GAGTG
134155524,"[CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, AGTTTTATGACTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, TAAACATTCAAGTTTTATGACTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, GAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, ATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT, TATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT]","[ATGGACTGATGCTTCCTGGTCCCTTCTCCTGCAGACTGAGATGACATCCTGTGCAGGGGTGATGTCAAGGGGGA, ATGGACTGATGCTTCCTGGTCCCTTCTCCTGCAGACTGAGATGACATCCTGTGCAGGGGTGATGTCAAGGGGGAGCAGGT, ATGGACTGATGCTTCCTGGTCCCTTCTCCTGC, ATGGACTGATGCTTCTTGGTCCCTTCTCCTGCAGACTGAGA, ATGGACTGATGCTTCCTGGTCCCTTCTCCTGCAGACTGAG]",AT


In [11]:
grouped_queries['combined'] = grouped_queries['first'] + grouped_queries['last']
temp = grouped_queries[['combined', 'homology']].explode('combined')
homologies = dict(zip(temp['combined'], temp['homology']))
new_awesomesauce_df['homology'] = new_awesomesauce_df['new_awesomesauce_query'].apply(lambda x: homologies[x])
new_awesomesauce_df = new_awesomesauce_df[['break_chrom1', 'break_pos1', 'break_chrom2', 'break_pos2', 'AA_homology_seq', 'homology', 'query_name', 'split','query_chrom', 'query_pos', 'query_end', 'query_orientation', 'query_cigar', 'query_aln_sub', 'query_aln_full']]
new_awesomesauce_df = new_awesomesauce_df.reset_index(drop=True)
new_awesomesauce_df.to_csv('final.tsv', sep='\t')
new_awesomesauce_df

,break_chrom1,break_pos1,break_chrom2,break_pos2,AA_homology_seq,homology,query_name,split,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub,query_aln_full
0,chr13,92474011,chr13,92475758,AA,TT,57596,True,chr13,92473942,92474011,-,70M30S,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,AGACGTAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTT
1,chr13,92474011,chr13,92475758,AA,TT,57596,True,chr13,92475727,92475758,+,32M68H,AAATTAATGCTCCCACTTCAACTTAAAGAATT,AAATTAATGCTCCCACTTCAACTTAAAGAATT
2,chr13,92474011,chr13,92475758,AA,TT,83594,True,chr13,92475722,92475758,-,37M63H,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT,AAAGTAAATTAATGCTCCCACTTCAACTTAAAGAATT
3,chr13,92474011,chr13,92475758,AA,TT,83594,True,chr13,92473947,92474011,+,65M35S,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAA,TAATGTGATTTCTTCTAATCTATTCATATGATTATTATATTAATGGATGCTAACTAATCATCAAATTCTTTAAGTTGAAGTGGGAGCATTAATTTACTTT
4,chr13,81088033,chr13,81469935,AAC,AAC,78137,True,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG
5,chr13,81088033,chr13,81469935,AAC,AAC,78137,True,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC
6,chr13,81088033,chr13,81469935,AAC,AAC,60717,True,chr13,81469935,81469972,+,62H38M,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC,GTTTCTTTGGTCTTTATTCTTCTGTTTTTTTCATTAAC
7,chr13,81088033,chr13,81469935,AAC,AAC,60717,True,chr13,81088033,81088097,-,35S65M,AACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG,GTTAATGAAAAAAACAGAAGAATAAAGACCAAAGAAACTTTTATCCTCTCTCATGGGGTGCATTACTTTCCAGTCTGGTACATTGTCTAGAAAAGTGTCG
8,chr9,134155524,chr13,108661411,AT,AT,77319,False,chr13,108661338,108661411,+,74M26S,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,CTTTGAGTGAAGTATTTAGGACTTGCTTCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCTTCTC
9,chr9,134155524,chr13,108661411,AT,AT,60793,False,chr13,108661334,108661411,+,78M22S,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAAT,ATGACTTTGAGTGAAGTATTTAGGACTTGCATCTATTGAGGCTATTTTACCTTCCTGCTTTGTCTTGAAAATTATAATGGACTGATGCTTCCTGGTCCCT


In [12]:
# new_awesomesauce_df["homology_cut"] = new_awesomesauce_df.apply(lambda x: x['query_aln_sub'][:7] if x["break_start"] else x['query_aln_sub'][-7:], axis=1)
# new_awesomesauce_df["wrong_dog"] = new_awesomesauce_df.apply(lambda x: str(x['AA_homology_seq']) not in x['homology_cut'], axis=1)
# new_awesomesauce_df.loc[new_awesomesauce_df["wrong_dog"] == True]

In [13]:
df = pd.read_csv('minor_errors.tsv', sep='\t')
df

,Unnamed: 0,break_chrom1,break_pos1,break_chrom2,break_pos2,break_orientation,AA_homology_len,AA_homology_seq,query_name,query_short,split,proper_pair,read_num,query_chrom,query_pos,query_end,query_orientation,query_cigar,query_aln_sub
0,13,chr9,133607147,chr22,23632742,-+,5.0,GAGTG,HWI-ST1113:353:H99R3ADXX:2:2212:8483:98862,98862,True,Discordant,2,chr9,133607151,133607180,+,70H30M,GGGTTTTATCAGCTTCCATACCCAAACAGA
1,14,chr9,133607147,chr22,23632742,-+,5.0,GAGTG,HWI-ST1113:353:H99R3ADXX:2:2212:8483:98862,98862,True,Discordant,2,chr22,23632672,23632740,+,69M31S,CAGGGTGTGGGGAAACAGGGAGGTTGTTCAGATGACCACGGGACACCTTTGACCCTGGCCGCTGTGGAG
2,80,chr9,133607147,chr22,23632742,-+,5.0,GAGTG,HWI-ST1113:353:H99R3ADXX:2:1115:16416:31425,31425,False,Concordant,2,chr22,23632371,23632458,+,12S88M,GTATGTTTTTAGCCAAAGACACTCGCTTACCTTGTGCCAGGCAGATGGAAGCCCCATAGTGTCCACCGGAAGGGTGATTTTTAAGCAG
3,152,chr9,134155524,chr13,108661411,++,2.0,AT,HWI-ST1113:353:H99R3ADXX:2:2109:4740:80488,80488,False,Concordant,1,chr9,134155367,134155426,+,40S60M,ACTAGGAAGTCCTACCAACAGAGGCGTAGGAGCGCCCACATGCAGAGCACTTGCAAAGTA
4,153,chr9,134155524,chr13,108661411,++,2.0,AT,HWI-ST1113:353:H99R3ADXX:2:2109:4740:80488,80488,False,Concordant,2,chr9,134155452,134155515,-,64M36S,CCCCCTTGACATCACCAAAGCCCAGGACGTCATCTCAGTCTGCCGGAGAAGGGACCAGGAAGCA
5,192,chr13,81088033,chr13,81469935,--,3.0,AAC,HWI-ST1113:353:H99R3ADXX:2:2204:3704:15030,15030,True,Discordant,1,chr13,81469984,81470036,+,53M47S,ATAAACTATTTATACACATACACACGGATGTTCCATATGAGTCTGAAATAAAA
6,193,chr13,81088033,chr13,81469935,--,3.0,AAC,HWI-ST1113:353:H99R3ADXX:2:2204:3704:15030,15030,True,Discordant,1,chr13,81470259,81470311,-,53M47H,CTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT
7,231,chr13,81088033,chr13,81469935,--,3.0,AAC,HWI-ST1113:353:H99R3ADXX:2:1203:15872:69813,69813,False,Discordant,1,chr13,81470218,81470311,+,94M6S,ATGCCTGAATAATATTGATATTAGTTATTTGTAACTTGTATCTCTGTTTTAAAAAGTTTATCTATATGTCTTGCCACAGATTTATTACTTTTAT
8,241,chr13,81088033,chr13,81469935,--,3.0,AAC,HWI-ST1113:353:H99R3ADXX:2:1213:17970:31423,31423,False,Concordant,1,chr13,81469796,81469870,+,75M25S,CTTGCCTCATGCCTGGATGAGGGGTAAGAGACAGGAAACAGGAGGGCAGGAGATTGTCAGAGAGAAACTGCTTTT
9,274,chr13,81109414,chr13,90438707,+-,3.0,TGG,HWI-ST1113:353:H99R3ADXX:2:1105:1237:14946,14946,False,Concordant,2,chr13,81109280,81109377,+,2S98M,TGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGCGCGTGCCCATGTGCCTATGTGTTTATCTCCACAGTGCAATGTATTGGTTACATAATAGC
